In [1]:
import sqlite3
import pandas as pd

# Connect to KBO database
conn = sqlite3.connect(r'./data/kbo_database.db')

In [2]:
# Run a test query to check the connection
query = """
    SELECT *
    FROM enterprise
    LIMIT 5;
"""
enterprise_df  = pd.read_sql_query(query, conn)
enterprise_df.head()

,EnterpriseNumber,Status,JuridicalSituation,TypeOfEnterprise,JuridicalForm,JuridicalFormCAC,StartDate
0,0200.065.765,AC,0,2,416.0,None,09-08-1960
1,0200.068.636,AC,0,2,417.0,None,16-02-1923
2,0200.171.970,AC,0,2,116.0,None,01-01-1968
3,0200.245.711,AC,12,2,116.0,None,01-01-1922
4,0200.305.493,AC,0,2,416.0,None,19-03-1962
